# Cервис для чтения книг по подписке

**Цель работы:** проанализировать базу данных и сформулировать ценностное предложение для нового продукта.

**Описание данных:** 

Таблица books:

* book_id — идентификатор книги;
* author_id — идентификатор автора;
* title — название книги;
* num_pages — количество страниц;
* publication_date — дата публикации книги;
* publisher_id — идентификатор издателя.

Таблица authors:

* author_id — идентификатор автора;
* author — имя автора.

Таблица publishers:

* publisher_id — идентификатор издательства;
* publisher — название издательства;

Таблица ratings:

* rating_id — идентификатор оценки;
* book_id — идентификатор книги;
* username — имя пользователя, оставившего оценку;
* rating — оценка книги.

Таблица reviews:

* review_id — идентификатор обзора;
* book_id — идентификатор книги;
* username — имя автора обзора;
* text — текст обзора.

**План работы**

* Исследование данных
* Анализ данных:
 * Подсчет количества книг, вышедших после 1 января 2000 года
 * Подсчет количество обзоров и среднюю оценку для каждой книги
 * Определение издательства, которое выпустило наибольшее число книг толще 50 страниц 
 * Определение автора с самой высокой средней оценкой книг, имеющих 50 и более оценок
 * Подсчет среднего количества обзоров от пользователей, которые поставили больше 50 оценок
 * Выводы



## Исследованиe данных

In [1]:
import pandas as pd
from sqlalchemy import create_engine
# устанавливаем параметры
#db_config не указываем, чтобы не раcкрывать доступ к базе 

#connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 #db_config['pwd'],
 #db_config['host'],
 #db_config['port'],
 #db_config['db'])
# сохраняем коннектор
#engine = create_engine(connection_string, connect_args={'sslmode':'require'})


In [2]:
table_list=['books', 'authors', 'ratings', 'publishers', 'reviews']


def head(name):
    
    query = f'''SELECT * FROM {name} '''
    print('Первые 5 строк из датасета', name, ':')
    
    name= pd.io.sql.read_sql(query, con = engine) 

    name.to_csv('log_raw.csv', index = False)

    return name.head(5)


for name in table_list:    
    display(head(name))

Первые 5 строк из датасета books :


,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


Первые 5 строк из датасета authors :


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


Первые 5 строк из датасета ratings :


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


Первые 5 строк из датасета publishers :


,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


Первые 5 строк из датасета reviews :


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


## Анализ данных

Посчитаем сколько книг вышло после 1 января 2000 года:

In [3]:
query = '''
           SELECT COUNT(book_id)
           FROM books
           WHERE publication_date >= '2000-01-01'
        '''

pd.io.sql.read_sql(query, con = engine) 


,count
0,821


Вышло 821 книги после 1 января 2000 включительно

Далее посчитаем количество обзоров и среднюю оценку для каждой книги:

In [4]:
query = '''
           SELECT books.book_id,
                  COUNT(DISTINCT reviews.review_id) AS review_count,
                  ROUND(AVG(ratings.rating),2) AS avg_rating
                  
           FROM books
           LEFT JOIN ratings ON books.book_id=ratings.book_id
           LEFT JOIN reviews ON ratings.book_id=reviews.book_id
           GROUP BY books.book_id
           
           
           
        '''

pd.io.sql.read_sql(query, con = engine) 


,book_id,review_count,avg_rating
0,1,2,3.67
1,2,1,2.50
2,3,3,4.67
3,4,2,4.50
4,5,4,4.00
...,...,...,...
995,996,3,3.67
996,997,3,3.40
997,998,4,3.20
998,999,2,4.50


Определим издательство, которое выпустило наибольшее число книг толще 50 страниц:

In [5]:
query = '''
           SELECT publishers.publisher,
           COUNT(books.book_id) AS count_books
                  
           FROM publishers
           LEFT JOIN books ON publishers.publisher_id=books.publisher_id
           WHERE books.num_pages>50
           GROUP BY publishers.publisher
           ORDER BY count_books DESC
           LIMIT 1
        '''

pd.io.sql.read_sql(query, con = engine) 

,publisher,count_books
0,Penguin Books,42


Penguin Books -издательство, выпустившее наибольшее число книг толще 50 страниц

Далее определим автора с самой высокой средней оценкой книг, будем учитывать только книги с 50 и более оценками:

In [6]:
query = '''
           SELECT authors.author,
                  ROUND(AVG(ratings.rating),2) AS avg_rating
                  
                  
           FROM authors
           LEFT JOIN books ON authors.author_id=books.author_id
           LEFT JOIN ratings ON books.book_id=ratings.book_id
           WHERE books.book_id IN (SELECT book_id
                    FROM ratings
                    GROUP BY book_id
                    HAVING COUNT(rating_id) >= 50)
           GROUP BY authors.author
           ORDER BY avg_rating DESC
           LIMIT 1
        '''

pd.io.sql.read_sql(query, con = engine) 

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.29


Далее посчитаем среднее количество обзоров от пользователей, которые поставили больше 50 оценок:

In [7]:
query = '''WITH 

table1 AS (SELECT username,
          COUNT(rating)
          FROM ratings
          GROUP BY username
          HAVING COUNT(rating)>50),

table2 AS (SELECT username,
          COUNT(review_id) AS count_review
          FROM reviews
          GROUP BY username)
SELECT ROUND(AVG(count_review), 2)
FROM table1
LEFT JOIN table2 ON table1.username=table2.username
'''
pd.io.sql.read_sql(query, con = engine)   

,round
0,24.33


Cреднее количество обзоров от пользователей, которые поставили больше 50 оценок, равняется 24,33

**Выводы:**
* Вышло 821 книги после 1 января 2000 включительно
* Penguin Books -издательство, выпустившее наибольшее число книг толще 50 страниц
* С учетом книг с 50 и более оценками, автором с самой высокой средней оценкой книг является J.K. Rowling/Mary GrandPré	
* Cреднее количество обзоров от пользователей, которые поставили больше 50 оценок, равняется 24,33

Таким образом наиболее популярными являются  книги от J.K. Rowling/Mary GrandPré и от издательства Penguin Books